<a href="https://colab.research.google.com/github/Strugoeden121/DataScience236006-Final/blob/main/236003_Final_Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import combinations

In [ ]:
# Load the Adult Income dataset (replace with actual file path if needed)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns_for_underepresented = ['age-binned', 'workclass', 'education', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'continent']

columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
           'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
           'hours-per-week', 'native-country', 'income']


# Load the dataset into a DataFrame
df = pd.read_csv(url, names=columns, na_values=' ?', skipinitialspace=True)

# Drop rows with missing values
df = df.dropna()

In [ ]:
#binning
# Step 1: Binning the 'age' column into decades
df['age-binned'] = pd.cut(df['age'], bins=[0, 20, 30, 40, 50, 60, 70, 80, 100],
                          labels=['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-100'])

# Step 2: Mapping 'native-country' to continents
def map_country_to_continent(country):
    continent_map = {
        'North America': ['United-States', 'Canada', 'Mexico','Outlying-US(Guam-USVI-etc)',],
        'Central America': [ 'Puerto-Rico', 'Honduras', 'Cuba', 'Dominican-Republic', 'Jamaica', 'El-Salvador', 'Haiti', 'Trinadad&Tobago'],
        'Asia': ['China', 'India', 'Japan', 'Philippines', 'Vietnam', 'South'],
        'Europe': ['England', 'Germany', 'Italy', 'Poland', 'Portugal', 'France', 'Greece', 'Ireland'],
        'South America': ['Columbia', 'Ecuador', 'Peru'],
        'Africa': ['South-Africa', 'Egypt'],

    }

    for continent, countries in continent_map.items():
        if country in countries:
            return continent
    return 'Other'

df['continent'] = df['native-country'].apply(map_country_to_continent)

In [ ]:
# Function to calculate the proportion of each group in the dataset based on a variable threshold
def calculate_group_proportions(df, columns, threshold):
    proportions = {}
    for col in columns:
        group_counts = df[col].value_counts(normalize=True)
        proportions[col] = group_counts[group_counts < threshold]  # Identify groups under threshold
    return proportions


In [ ]:
# Single-column underrepresented groups with a 10% threshold
single_column_underrepresented = calculate_group_proportions(df, columns_for_underepresented, 0.20)

# Display single-column underrepresented groups
print("Single-column underrepresented groups:")
for feature, groups in single_column_underrepresented.items():
    if not groups.empty:
        print(f"Feature: {feature}")
        print(groups)
        print()


Single-column underrepresented groups:
Feature: age-binned
age-binned
51-60     0.126777
0-20      0.074015
61-70     0.055035
71-80     0.013544
81-100    0.003040
Name: proportion, dtype: float64

Feature: workclass
workclass
Self-emp-not-inc    0.078038
Local-gov           0.064279
?                   0.056386
State-gov           0.039864
Self-emp-inc        0.034274
Federal-gov         0.029483
Without-pay         0.000430
Never-worked        0.000215
Name: proportion, dtype: float64

Feature: education
education
Bachelors      0.164461
Masters        0.052916
Assoc-voc      0.042443
11th           0.036086
Assoc-acdm     0.032769
10th           0.028654
7th-8th        0.019840
Prof-school    0.017690
9th            0.015786
12th           0.013298
Doctorate      0.012684
5th-6th        0.010227
1st-4th        0.005160
Preschool      0.001566
Name: proportion, dtype: float64

Feature: marital-status
marital-status
Divorced                 0.136452
Separated                0.031479


In [ ]:
# Multi-column underrepresented groups with more diverse thresholds
def find_joint_underrepresented_groups(df, feature_combinations, threshold):
    joint_groups = []
    for due in feature_combinations:
        grouped = df.groupby(list(due)).size() / len(df)
        underrepresented = grouped[grouped < threshold]
        if not underrepresented.empty:
            joint_groups.append((due, underrepresented))
    return joint_groups


In [ ]:
feature_combinations = [combinations(columns_for_underepresented, 2)]  # 2 columns
flat_combinations = [item for sublist in feature_combinations for item in sublist]
joint_underrepresented_groups = find_joint_underrepresented_groups(df, flat_combinations, 0.10)
print(joint_underrepresented_groups)

<ipython-input-24-a9b566fa2d89>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(list(due)).size() / len(df)
<ipython-input-24-a9b566fa2d89>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(list(due)).size() / len(df)
<ipython-input-24-a9b566fa2d89>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(list(due)).size() / len(df)
<ipython-input-24-a9b566fa2d89>:5:

[(('age-binned', 'workclass'), age-binned  workclass       
0-20        ?                   0.011793
            Federal-gov         0.000553
            Local-gov           0.001443
            Never-worked        0.000154
            Private             0.056325
                                  ...   
81-100      Private             0.001474
            Self-emp-inc        0.000215
            Self-emp-not-inc    0.000430
            State-gov           0.000031
            Without-pay         0.000000
Length: 69, dtype: float64), (('age-binned', 'education'), age-binned  education   
0-20        10th            0.006357
            11th            0.012837
            12th            0.004300
            1st-4th         0.000123
            5th-6th         0.000369
                              ...   
81-100      HS-grad         0.001013
            Masters         0.000215
            Preschool       0.000000
            Prof-school     0.000123
            Some-college    0.00039

In [ ]:
#Plot Underrepresented